# Project in TNM098 VT 2018, VAST Challenge MC2 2015

This project will explore data containing phone calls between people in an amusement park. The questions that will be answered are:

1. Identify those IDs that stand out for their large volumes of communication. For each of these IDs
  1. Characterize the communication patterns you see.
  2. Based on these patterns, what do you hypothesize about these IDs?

Please limit your response to no more than 4 images and 300 words.

2. Describe up to 10 communications patterns in the data. Characterize who is communicating, with whom, when and where. If you have more than 10 patterns to report, please prioritize those patterns that are most likely to relate to the crime.

Please limit your response to no more than 10 images and 1000 words.

#### My approach:
To start solving the first problem, I would filter out all communication from the IDs with large volumes. From that I would try to visualize the following:
* With who are these persons are communicating with?
* From what places?
* Where is the recipient located?
* What time of the day?
* Do the recipients communicate with each other?
* And more possible properties of the communication data that are relevant


First, start with imports.

In [1]:
import numpy as np
import pandas as pd
import nvd3
import ipywidgets as widgets
from IPython.display import display, HTML
import datetime
import time

#import matplotlib.pyplot as plt
#import pylab
#import plotly.plotly as py

loaded nvd3 IPython extension
run nvd3.ipynb.initialize_javascript() to set up the notebook
help(nvd3.ipynb.initialize_javascript) for options


Read in the data for Friday.
Pandas series is a vector and a dataframe is a matrix of series.

In [2]:
data_Fri = pd.read_csv("data/comm-data-Fri.csv")
#type(data_Fri): pandas.core.frame.DataFrame
#data_Fri.head(): gives 5 first elements


#	Timestamp 			from 		to 			location
#0 	2014-6-06 08:03:19 	439105 		1053224 	Kiddie Land
#1 	2014-6-06 08:03:19 	439105 		1696241 	Kiddie Land
#2 	2014-6-06 08:03:19 	439105 		580064 		Kiddie Land
#3 	2014-6-06 08:03:19 	439105 		1464748 	Kiddie Land
#4 	2014-6-06 08:03:47 	1836139 	1593258 	Entry Corridor


Find out which senders stand out for their large communication volumes. Pick the five largest.

In [3]:
#Count occurrences for each sender
from_data = data_Fri['from']
to_data = data_Fri['to']
send_counts = from_data.value_counts()
receive_counts = to_data.value_counts()

#print(send_counts)
#print(receive_counts)

#send_average = send_counts.mean()
#receive_average = receive_counts.mean()

too_low_senders = send_counts[send_counts <= 1650].index
highest_senders = send_counts.drop(too_low_senders)

too_low_receivers = receive_counts[receive_counts <= 1350].index
# Remove 'external' from receivers
too_low_receivers = too_low_receivers.append(pd.Index(['external']))
highest_receivers = receive_counts.drop(too_low_receivers)

#print(highest_senders)
#print(highest_receivers)

Different ways to access the data in the DataFrame

In [4]:
#data_Fri.loc[0:1, ['to', 'location']]
#data_Fri['from']

In [5]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)
#display(w)

Dropdown(description='Number:', index=1, options=('1', '2', '3'), value='2')

Use nvd3 to construct the plot, and export it to view as html.

In [10]:
nvd3.ipynb.initialize_javascript(use_remote=True)
np.random.seed(100)

chart_type = 'discreteBarChart'
chart = nvd3.discreteBarChart(name=chart_type, height=700, width=1500)

ydata = highest_senders.to_list()
xdata = list(highest_senders.keys())

chart.add_serie(y=ydata, x=xdata)
chart.buildhtml()
chart_html = chart.htmlcontent

# The chart in html code
#chart_html

# Show the plot
display(HTML(chart_html))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Since 1278894 is clearly the biggest sender and receiver, a closer look will be made on its communication patterns.

In [7]:
# Function for fixing date format in data to be able to compare dates
def convert_to_datetime(str_date):
    #print(str_date)
    date = str_date[0:5] + "0" + str_date[5:]
    return datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")

In [8]:
# Get all data where 1278894 is the sender
sender_1278894 = data_Fri[data_Fri['from'] == 1278894]
#print(sender_1278894.sort_values('Timestamp'))
#print(sender_1278894)
#for label, content in sender_1278894.iteritems():
#    print('label:', label)
#    print('content:', content, sep='\n')
#timestamps = sender_1278894['Timestamp']
#for index, data in timestamps.iteritems():
#    data = convert_to_datetime(data)
#print(timestamps)
# 

counts = sender_1278894['Timestamp'].value_counts()
sorted_counts = counts.sort_index()


#sorted_counts.to_list()
#list(sorted_counts.keys())

In [9]:
from nvd3 import stackedAreaChart
chart = stackedAreaChart(name='stackedAreaChart', x_is_date=True, date_format="%d %b %Y", height=700, width=1500)

# Datetimes on x-axis
xdata = [convert_to_datetime(x) for x in list(sorted_counts.keys())]
xdata = [time.mktime(s.timetuple()) * 1000 for s in xdata]

ydata = sorted_counts.to_list()
#ydata2 = [8, 20, 16, 12, 20, 28, 28]

extra_serie = {"tooltip": {"y_start": "There is ", "y_end": " min"}}
chart.add_serie(name="Serie 1", y=ydata, x=xdata, extra=extra_serie)
#chart.add_serie(name="Serie 2", y=ydata2, x=xdata, extra=extra_serie)
chart.buildhtml()

chart_html = chart.htmlcontent

# Show the plot
display(HTML(chart_html))